In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class thewhisky(scrapy.Spider):
    name = 'the_whisky'
    allowed_domains = ['www.thewhiskyexchange.com']
    start_urls = ['https://www.thewhiskyexchange.com/c/305/rest-of-the-world-whisky?pg=1']
    custom_settings = {
       'FEED_URI' : 'tmp/the_whisky.csv'
   }
    
    def parse(self,response):
        for i in range(1,14):
            products = response.xpath("//li[@class='product-grid__item']")
            for product in products:
                link = product.css('a.product-card').attrib['href']
                url = response.urljoin(link)
                yield scrapy.Request(url = url, callback= self.parse_details)
         #pagination   
            next_page = f'https://www.thewhiskyexchange.com/c/305/rest-of-the-world-whisky?pg={i}'
            yield scrapy.Request(url = next_page, callback=self.parse)
            
    def parse_details(self,response):
        name = response.css('h1.product-main__name::text').extract_first().strip()
        price = response.css('p.product-action__price::text').extract_first().strip()
        unit_price = response.css('p.product-action__unit-price::text').extract_first().strip()
        desc = response.css('div.product-main__description p::text').extract_first().strip()
        image = response.css('img.product-main__image').attrib['src']
        yield {
            'Name':name,
            'Price':price,
            'Unit price':unit_price,
            'Description':desc,
            'Image':image
        }

In [3]:
process = CrawlerProcess()
process.crawl(thewhisky)
process.start()

2022-07-13 23:35:02 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-07-13 23:35:02 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-8.1-6.3.9600-SP0
2022-07-13 23:35:02 [scrapy.crawler] INFO: Overridden settings:
{}
2022-07-13 23:35:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-07-13 23:35:02 [scrapy.extensions.telnet] INFO: Telnet Password: 49fa85252debb845
2022-07-13 23:35:02 [py.warnings] WARNING: C:\Users\Khaliq\anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:289: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2022-07-